# Lab 4: Data-Driven Optimisation

The purpose of this lab session is to study in more depth optimisation notions and use them to model and solve optimisation programs. Recall that in a previous lecture we discussed the different formats of *linear programs* and how to switch from one formutation to another. 

## 1. Verifying the model design (by revisiting tasks from lab 3)

In the previous lab session, we discussed two examples where you were asked to model them with linear programs and solve them using python libraries.

We begin by revisiting these two examples and checking their solution. Please check *lab3-files* and in particular the *How to build a model* file.  

**Example 1 (Task 5 in Lab 3):** Click Details to recall the example.

<details>
A company will face the following cash requirements in the next eight quarters (positive entries
represent cash needs while negative entries represent cash surpluses). 

| Q1 | Q2 | Q3 | Q4 | Q5 | Q6 | Q7 | Q8 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 100 | 500 | 100 | -600 | -500 | 200 | 600 | -900 |

The company has three borrowing possibilities.
<ul>
<li> a 2-year loan available at the beginning of Q1, with a 1% interest per quarter.
<li> The other two borrowing opportunities are available at the beginning of every quarter: a 6-
month loan with a 1.8% interest per quarter, and a quarterly loan with a 2.5% interest for the
quarter.
</ul>
    
Any surplus can be invested at a 0.5% interest per quarter. Formulate a linear program that maximises
the wealth of the company at the beginning of Q9. Solve the problem using **linprog** and interpret the
solution.
</details>

**Task 1**: Please see below for the solution and check file *How to build the model* in the *lab3-files* folder. See if the model and the implementation makes sense to you.

In [ ]:
import numpy as np
from scipy.optimize import linprog

# I generate the vector of coefficients in the objective function and initialize the matrix of contraint coefficients
c = np.concatenate([np.zeros(21), [-1]]) # Recall that linprog has a minimization objective, so we minimize -wealth
A = np.zeros((8,22))

# I create a row for each constraint
A[0] = np.concatenate([[1,1,1,-1],np.zeros(18)])
A[1] = np.concatenate([[-0.01, -0.018, -1.025, 1.005, 1, 1, -1],np.zeros(15)])
A[2] = np.concatenate([[-0.01, -1.018, 0, 0, -0.018, -1.025, 1.005, 1, 1, -1],np.zeros(12)])
A[3] = np.concatenate([[-0.01, 0, 0, 0, -1.018, 0, 0, -0.018, -1.025, 1.005, 1, 1, -1],np.zeros(9)])
A[4] = np.concatenate([[-0.01], np.zeros(6), [-1.018, 0, 0, -0.018, -1.025, 1.005, 1, 1, -1],np.zeros(6)])
A[5] = np.concatenate([[-0.01], np.zeros(9), [-1.018, 0, 0, -0.018, -1.025, 1.005, 1, 1, -1],np.zeros(3)])
A[6] = np.concatenate([[-0.01], np.zeros(12), [-1.018, 0, 0, -0.018, -1.025, 1.005, 1, -1, 0]])
A[7] = np.concatenate([[-1.01], np.zeros(15), [-1.018, 0, 0, -1.025, 1.005, -1]])

b = [100, 500, 100, -600, -500, 200, 600, -900]

# It is now time to solve. All variables are constrained to be non-negative, so I don't need to specify the bounds; this is the default
res = linprog(c, A_eq=A, b_eq=b, method='revised simplex', options={"disp": True})

# Print the solution. Recall that the wealth is the last variable. 
res.x

**Example 2 (Task 6 in Lab 3):**  Click Details to recall the example.

<details>
Consider a restaurant that is open seven days a week. Based on past experience, the number of workers needed on a particular day is given as follows: 

| Mon | Tue | Wed | Thu | Fri | Sat | Sun | 
| --- | --- | --- | --- | --- | --- | --- |
| 14 | 13 | 15 | 16 | 19 | 18 | 11 |

Every worker works five days in a week and has two days off in the following pattern: three days work, one day off, two days work, one day off. So, there are workers working on Mon-Tue-Wed-Fri-Sat, other workers on Tue-Wed-Thu-Sat-Sun, etc. How can we minimize the number of workers that staff the restaurant? 
</details>

**Task 2**: Please see below for the solution and check file How to build the model in the lab3-files folder. See if the model and the implementation makes sense to you.

In [ ]:
import numpy as np
from scipy.optimize import linprog

# I generate the vector of coefficients in the objective function 
c = np.array([1,1,1,1,1,1,1])
A = np.zeros((7,7))

# I create a row for each constraint
A[0] = np.array([-1,0,-1,-1,-1,0,-1])
A[1] = np.array([-1,-1,0,-1,-1,-1,0])
A[2] = np.array([0,-1,-1,0,-1,-1,-1])
A[3] = np.array([-1,0,-1,-1,0,-1,-1])
A[4] = np.array([-1,-1,0,-1,-1,0,-1])
A[5] = np.array([-1,-1,-1,0,-1,-1,0])
A[6] = np.array([0,-1,-1,-1,0,-1,-1])
b = np.array([-14, -13, -15, -16, -19, -18, -11])

# It is now time to solve. All variables are constrained to be non-negative, so I don't need to specify the bounds; this is the default
res = linprog(c, A_ub=A, b_ub=b, options={"disp": True})

# Print the solution. Recall that the wealth is the last variable. 
res.x

It is clear from the solution that our model did **not** take into account the fact that the number of workers must be an *integer* number. Hence, in last week's lab  we discussed integer linear programming (see Section 5 in Lab 3).

## 2. Testing our understanding on Duality

### Complementary Slackness and Optimality - An example

In the most recent lecture, we discussed duality and, among other terrifying things, complementary slackness. Let's see an example.

\begin{eqnarray*}
\max & \hspace{1cm}& x_1 - 2x_2 +3x_3 \\
\text{subject to} & \hspace{1cm}&x_1+x_2-2x_3\leq 1\\
& \hspace{1cm}&2x_1-x_2-3x_3 \leq 4\\
& \hspace{1cm}&x_1+x_2+5x_3 \leq 2\\
& \hspace{1cm}&x_1, x_2, x_3 \geq 0
\end{eqnarray*}

Let's build its dual Linear Program. Observe that the LP above is a maximisation one. This should not be a problem, as we have seen that the primal and the dual LPs are two "siblings" where one LP is a minimisation problem while the other is a maximisation one. 

For variable $x_1$ of the primal LP, we obtain the constraint for the dual LP that $y_1+2y_2+y_3 \geq 1$. This is because we multiply each constraint above with a coefficient $y_i$ that is non-negative, and we wish the total coefficient of $x_1$ in this weighted combination to be bounded from below by the coefficient of $x_1$ in the objective function of the primal LP.

It is the same reasoning that gives us the additional constraints $y_1 - y_2 + y_3 \geq -2$ and $-2y_1-3y_2+5y_3 \geq 3$.
Additionally, we require that $y_1, y_2, y_3 \geq 0$.

The objective function of the dual LP is now a *minimisation* function and is actually to

\begin{eqnarray*}
\min & \hspace{1cm}& y_1 + 4y_2 +2y_3 \\
\end{eqnarray*}

Overall, we obtain that the dual LP is 
\begin{eqnarray*}
\min & \hspace{1cm}& y_1 + 4y_2 +2y_3 \\
\text{subject to} & \hspace{1cm}&y_1+2y_2+y_3 \geq 1\\
& \hspace{1cm}&y_1 - y_2 + y_3 \geq -2\\
& \hspace{1cm}&-2y_1-3y_2+5y_3 \geq 3\\
& \hspace{1cm}&y_1, y_2, y_3 \geq 0
\end{eqnarray*}

Say someone tells us that $x^*_1 = 9/7, x^*_2 = 0, x^*_3 = 1/7$ is an optimal solution for the primal LP (the maximisation problem). Let’s try to verify that claim.

At least those values satisfy the constraints! The objective value of the primal LP for these values becomes $12/7$.

Now let’s see what complementary slackness would tells us about an optimal solution $y^*_1, y^*_2, y^*_3$ of the dual. 
  
Because $x^*_1$ and $x^*_3$ are non-zero, the first and third constraints of the dual must be tight!
That is $y^*_1+2y^*_2+y^*_3 = 1$ and $-2y^*_1-3y^*_2+5y^*_3 = 3$
 
That’s only two equations for three unknowns! 

But checking the primal, we see that the alleged optimal solution shows that the second constraing in the primal is *not* tight (that is $2x^*_1-x^*_2-3x^*_3 < 4$). By complementary slackness again, this implies that $y^*_2 = 0$. Plugging that in we get the following system:
$y^*_1 +y^*_3 = 1$ and $-2y^*_1 +5y^*_3 = 3$ whose only solution is $y^*_1 = 2/7$ and $y^*_3 = 5/7$; recall that we already established that $y^*_2 = 0$.

Observe that $y^*_1+4y^*_2+2y^*_3 = 12/7$. 
This means that the primal and the dual appear to have the same value and one would think that we are done and we have verified the claim. <b>That's not entirely correct!</b>

We still need to check whether the "optimal" values for the dual satisfy the contraints of the dual. One can check that this is indeed the case and now, finally, we can verify the claim.

## 3. Using different solvers

### 3a. OPTIONAL READING: Excel SOLVER
We now work with a solver outside Python, namely the solver embedded in the well-known Excel application. Please check the folder *lab4-files* in the CF969 moodle page.

**Task 5:** Please open the file *How to add Solver to Excel.pdf* and work through the instructions to get the Excel SOLVER add-in to work. Using the instructions, run Excel’s SOLVER on the example linear program (LP) described in the file *LP with Excel (model building).pdf*, to which *How to add Solver to Excel.pdf* also refers.

**Task 6:** Solve Example 2 (Task 6 in Lab 3) using Excel SOLVER. Recall that we now need to impose the constraint for an integral solution.

**Solution to Task 6**: Check file *task6-sol.xlsx*.

### 3b. Gurobi and Gurobipy

Gurobi is a powerful optimisation software. It is installed in the lab computers and we covered an example in the most recent lecture on how to use it. In case you wish to check it now, please visit slides 23 to 32 in file *CF969 - 3 - Data Driven Optimisation - Linear Programming.pptx* in the CF969 moodle page.

Apart from using Gurobi on its own, we have the option to invoke Gurobi through Python, by using the *gurobipy* package.

#### Installing Gurobipy

To install gurobipy, please see https://support.gurobi.com/hc/en-us/articles/360044290292-How-do-I-install-Gurobi-for-Python-

If your are using Horizon (https://csee-horizon.essex.ac.uk/portal/webclient/index.html), please open a command prompt and execute (while still in drive m:\) "m:\python -m pip install gurobipy" (without the quotes)


##### Solving an Integer Linear Program with Gurobipy

The following example shows how to use gurobipy to solve an integer linear program. Recall that in integer linear programs, each variable must be given an integer value. Such constraints arise quite naturally in many different settings.


#### Task Assignment Problem formulation

Consider three tasks that need to be completed: let's call them A, B, and C. 

Consider three users that, in principle, can do the tasks: 1, 2, and 3.

#### Data 

The ability to perform each task is as follows: User 1 can execute task A with quality 0.53, task B with quality 0.27 and task C with quality 0.13. Similarly, user 2 has 0.8 for task A, 0.47 for task B, and 0.67 for task C, and finally, user 3 has 0.53 for task A, 0.73 for task B, and 0.47 for task C.

**Assumption**: Only one user can be assigned to a task, and each user must execute at most one task.

#### Problem statement

Determine an assignment that ensures that each task is executed and each user executes at most one tasks in order to maximize the total quality of the tasks executed according to the assignment.

#### Decision variables

The decision variable $x_{r,\; j} = 1$ represents that user r is assigned to task j, and 0 otherwise, for  r=1,2,3 and 𝑗=1,2,3.

#### Constraints

#### Jobs constraints

For each task 𝑗=A,B,C, exactly one user from r=1,2,3 must be assigned.

Constraint (A=1): $x_{1,\; A} + x_{2,\; 1} + x_{3,\; 1} = 1$

Constraint (B=2): $x_{1,\; 2} + x_{2,\; 2} + x_{3,\; 2} = 1$

Constraint (C=3): $x_{1,\; 3} + x_{2,\; 3} + x_{3,\; 3} = 1$

#### Resources constraints

For each user r=1,2,3, at most one task from j=A,B,C can be assigned.

Constraint (user 1): $x_{1,\; 1} + x_{1,\; 2} + x_{1,\; 3}  \leq 1$

Constraint (user 2): $x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

Constraint (user 3): $x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

#### Objective function

The objective function is to maximize the total quality of the assignments while satisfying the user and task constraints.

$$
\max \; (53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) + (27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2})
+ (13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})
$$




In [ ]:
!pip install gurobipy --user
# import gurobi library
from gurobipy import *

## Data
The list R contains the id's of the three users: 1, 2, and 3. 

The list J contains the names of the tasks: A, B, and C.

In [ ]:
# resources and jobs sets
R = ['1', '2', '3']
J = ['A', 'B', 'C']

$r \in R$ index and set of users.

$j \in J$ index and set of tasks.

The following “multidict” function describes the quality score associated with each possible combination of a user and a task

In [ ]:
# matching score data
combinations, ms = multidict({
    ('1', 'A'): 53,
    ('1', 'B'): 27,
    ('1', 'C'): 13,
    ('2', 'A'): 80,
    ('2', 'B'): 47,
    ('2', 'C'): 67,
    ('3', 'A'): 53,
    ('3', 'B'): 73,
    ('3', 'C'): 47
})

The following function generates an empty model object “m” and takes the string “RAP” model name as its argument.

In [ ]:
# Declare and initialize model
m = Model('RAP')

## Decision variables

The decision variable $x_{r,\; j} = 1$ represents that user r is assigned to task j, and 0 otherwise, for $r \in R$ and $j \in J $.

The “addVars()” method defines the decision variables of the model object “m”.  

In [ ]:
# Create decision variables for the RAP model
x = m.addVars(combinations, vtype=GRB.INTEGER, name="assign")

## Task constraints

For each task 𝑗= A,B,C, exactly one user from r=1,2,3 must be assigned.

Constraint (A=1): $x_{1,\; 1} + x_{2,\; 1} + x_{3,\; 1} = 1$

Constraint (B=2): $x_{1,\; 2} + x_{2,\; 2} + x_{3,\; 2} = 1$

Constraint (C=3): $x_{1,\; 3} + x_{2,\; 3} + x_{3,\; 3} = 1$

The “addConstrs()” method defines the constraints of the model object “m”. 


In [ ]:
# create tasks  constraints
job = m.addConstrs((x.sum('*',j) == 1 for j in J), 'task')

## User constraints

For each user r=1,2,3, at most one task from r=1,2,3 can be assigned.

Constraint (1): $x_{1,\; 1} + x_{1,\; 2} + x_{1,\; 3}  \leq 1$

Constraint (2): $x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

Constraint (3): $x_{3,\; 1} + x_{3,\; 2} + x_{3,\; 3}  \leq 1$

The “addConstrs()” method defines the constraints of the model object “m”. 

In [ ]:
# create resources constraints
resource = m.addConstrs((x.sum(r,'*') <= 1 for r in R), 'user')

## Objective Function

The objective function is to maximize the total quality score of the assignments.

$$
Max \; (53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) + (27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2})
+ (13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})
$$

The “setObjective()” method defines the objective function of the model object “m”. 

In [ ]:
# The objective is to maximize total matching score of the assignments
m.setObjective(x.prod(ms), GRB.MAXIMIZE)

## Objective function
Notice that 
$$
(53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) = \sum_{r \; \in \; R} ms_{r,1}x_{r,1} \\
(27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2}) = \sum_{r \; \in \; R} ms_{r,2}x_{r,2} \\
(13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})  = \sum_{r \; \in \; R} ms_{r,3}x_{r,3}
$$

Hence, the objective function can be expressed as follows

$$
Max \; \sum_{j \; \in \; J} \sum_{r \; \in \; R} ms_{r,j}x_{r,j}
$$

In [ ]:
# save model for inspection
m.write('RAP.lp')

In [ ]:
# run optimization engine
m.optimize()

In [ ]:
def print_solution(model):
    for var in model.getVars():
        if abs(var.x) > 1e-6:
            print("{0}: {1}".format(var.varName, var.x))
    print('Total quality score: {0}'.format(model.objVal))
    return None

# display optimal values of decision variables
print_solution(m)   


**Task 7:** Solve Example 1 using gurobipy

**Solution to Task 7:** See the next snippet. Uncomment the line and run it.

In [ ]:
# %load ./sol7.py
import numpy as np
from gurobipy import *
from gurobipy import GRB

# Declare and initialize model
m = Model('Task 7')

# Add variables
x = m.addMVar(shape=22, vtype=GRB.CONTINUOUS, name="x")

# Create matrix A
A = np.zeros((8,22))

# I create a row for each constraint
A[0] = np.concatenate([[1,1,1,-1],np.zeros(18)])
A[1] = np.concatenate([[-0.01, -0.018, -1.025, 1.005, 1, 1, -1],np.zeros(15)])
A[2] = np.concatenate([[-0.01, -1.018, 0, 0, -0.018, -1.025, 1.005, 1, 1, -1],np.zeros(12)])
A[3] = np.concatenate([[-0.01, 0, 0, 0, -1.018, 0, 0, -0.018, -1.025, 1.005, 1, 1, -1],np.zeros(9)])
A[4] = np.concatenate([[-0.01], np.zeros(6), [-1.018, 0, 0, -0.018, -1.025, 1.005, 1, 1, -1],np.zeros(6)])
A[5] = np.concatenate([[-0.01], np.zeros(9), [-1.018, 0, 0, -0.018, -1.025, 1.005, 1, 1, -1],np.zeros(3)])
A[6] = np.concatenate([[-0.01], np.zeros(12), [-1.018, 0, 0, -0.018, -1.025, 1.005, 1, -1, 0]])
A[7] = np.concatenate([[-1.01], np.zeros(15), [-1.018, 0, 0, -1.025, 1.005, -1]])

# The right-hand-side constants
rhs = np.array([100, 500, 100, -600, -500, 200, 600, -900])

# Adding the constraints
m.addConstr(A @ x == rhs, name="c")

# Setting the objective function. Here, I can be flexible and define a maximization problem, unlike in linprog
obj = np.concatenate([[0]*21,[1]])
m.setObjective(obj @ x, GRB.MAXIMIZE)

m.optimize()
print(x.X)

## 4. Quadratic programming - Portfolio optimisation

So far we have considered linear programs, that is, the objective function was a linear function and so are the constraints. We now turn our attention to a more general class of programs, where the objective function can be a quadratic function. 

In a previous lecture we discussed Markowitz's [portfolio optimisation](https://en.wikipedia.org/wiki/Markowitz_model) problem. In that, we want to find the fraction of the portfolio to invest among a set of stocks that balances risk and return. It is a Quadratic Programming (QP) model with vector and matrix data for returns and risk, respectively.  

The discussion below is slightly adapted from https://www.gurobi.com/jupyter_models/portfolio-selection-optimization/.


## Model Formulation
### Parameters

We use the following notation:

- $\mathbf{\mu}$: n-element vector denoting the return for each stock
- $Q$: n x n matrix measuring the covariance among stocks. In the lecture, we modelled $q_{ij}$ as $q_{ij} = \rho_{ij}\sigma_i\sigma_j$, where $\rho_{ij}$ is the *correlation coefficient* and $\sigma_i, \sigma_j$ are the *standard deviations* for stocks $i$ and $j$ respectively.

There is one additional parameter when solving the model parametrically:

- R: target return


### Decision Variables
- $x \ge 0$: n-element vector where each element represents the fraction of the portfolio to invest in each stock

### Objective Function
Minimize the total risk, a convex quadratic function:

\begin{equation}
\min x^T Q x
\end{equation}

### Constraints

Allocate the entire portfolio: the total investments should be 1.0 (100%), where $\mathbf{1}$ is a unit vector (all 1's):

\begin{equation}
\mathbf{1}^T  x = 1
\end{equation}


Return: In the lecture we set a constraint that the target return should be at least $R$. When we solve the model parametrically for different return values $r$, we add an equality constraint on the target return:

\begin{equation}
\mathbf{\mu} x = R
\end{equation}

## Python Implementation
### Stock data
Use [yfinance](https://pypi.org/project/yfinance/) library to get the latest 2 years of _actual stock data_ from the 20 most profitable US companies, [according to Wikipedia](https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue#List_of_companies_by_profit).

In [1]:
%pip install gurobipy yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 7.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import yfinance as yf

stocks = ['BRK-A', 'AAPL', 'MSFT', 'JPM', 'GOOG', 'BAC', 'INTC', 'WFC',
          'C', 'VZ', 'PFE', 'JNJ', 'META', 'F', 'GS', 'XOM',
          'FNMA', 'T', 'UNH', 'V' ]

data = yf.download(stocks, period='2y')

/var/folders/nf/hk3ypf6j0bvd96jqrr9zn22m0000gn/T/ipykernel_43825/3325442938.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, period='2y')
[*********************100%***********************]  20 of 20 completed


### Compute return and covariance

Using the downloaded stock data, find $\mu$ (return), $Q$ (covariance) and standard deviation values for stock prices:

In [ ]:
import numpy as np

closes = np.transpose(np.array(data.Close)) # matrix of daily closing prices
absdiff = np.diff(closes)                   # change in closing price each day
reldiff = np.divide(absdiff, closes[:,:-1]) # relative change in daily closing price
mu = np.mean(reldiff, axis=1)            # mean price change
Q = np.cov(reldiff)                     # covariance (standard deviations)
std = np.std(reldiff, axis=1)               # standard deviation

## Minimise risk by solving QP model

In [ ]:
import gurobipy as gp
from gurobipy import GRB
from math import sqrt

# Create an empty model
m = gp.Model('portfolio')

# Add matrix variable for the stocks
x = m.addMVar(len(stocks))

# The objective is to minimise risk (squared).  This is modeled using the
# covariance matrix, which measures the historical correlation between stocks
portfolio_risk = x @ Q @ x
m.setObjective(portfolio_risk, GRB.MINIMIZE)

# Fix budget with an equality constraint
m.addConstr(x.sum() == 1, 'budget')

# Verify model formulation
m.write('portfolio_selection_optimization.lp')

# Optimize model to find the minimum risk portfolio
m.optimize()

**Task 8:** What have we computed above? Is there something "missing" from the quadratic program?
**Solution** Click on *Details* below to reveal it.

<details>
    There is no constraint on the expected return. In effect, we are computing the minimum risk portfolio without any interest in our return.
</details>

## Display minimum risk portfolio using Pandas

In [ ]:
import pandas as pd
minrisk_volatility = sqrt(m.ObjVal)
minrisk_return = mu @ x.X # the notation <variable name>.X returns the value of the variable
pd.DataFrame(data=np.append(x.X, [minrisk_volatility, minrisk_return]),
             index=stocks + ['Volatility', 'Expected Return'],
             columns=['Minimum Risk Portfolio'])

## Compute the [efficient frontier](https://en.wikipedia.org/wiki/Efficient_frontier)

Solve the QP parametrically to find the lowest risk portfolio for different expected returns.


In [ ]:
# Create an expression representing the expected return for the portfolio
portfolio_return = mu @ x
target = m.addConstr(portfolio_return == minrisk_return, 'target')

# Solve for efficient frontier by varying target return
frontier = np.empty((2,0))
for r in np.linspace(mu.min(), mu.max(), 25):
    target.rhs = r
    m.optimize()
    frontier = np.append(frontier, [[sqrt(m.ObjVal)],[r]], axis=1)

## Plot results
Use the matplot library to plot the optimized solutions, along with the individual stocks:

In [ ]:
import matplotlib.pyplot as plt
#plt.figure(figsize=(10,10))

fig, ax = plt.subplots(figsize=(10,8))

# Plot volatility versus expected return for individual stocks
ax.scatter(x=std, y=mu,
           color='Blue', label='Individual Stocks')
for i, stock in enumerate(stocks):
    ax.annotate(stock, (std[i], mu[i]))

# Plot volatility versus expected return for minimum risk portfolio
ax.scatter(x=minrisk_volatility, y=minrisk_return, color='DarkGreen')
ax.annotate('Minimum\nRisk\nPortfolio', (minrisk_volatility, minrisk_return),
            horizontalalignment='right')

# Plot efficient frontier
ax.plot(frontier[0], frontier[1], label='Efficient Frontier', color='DarkGreen')

# Format and display the final plot
ax.axis([frontier[0].min()*0.7, frontier[0].max()*1.3, mu.min()*1.2, mu.max()*1.2])
ax.set_xlabel('Volatility (standard deviation)')
ax.set_ylabel('Expected Return')
ax.legend()
ax.grid()
plt.show()